# Create Project
In this notebook, we learn how to create and setup the projects.

To keep it simple, we're going to create only two projects:

- Library called `lib`: E.g. a library static libraries and header files. This library has no further dependencies
- Application called `app`: An application that depends on `Library`. The build artefact of this application is e.g. an `.exe` file

## Preparation
As shown in a different notebook, we first have to build the infrastructure. This notebook assumes the reader has worked through the other notebook and thus therefore not explain anything here.

In [ ]:
from cicd_sim import *
bitbucket = Repos()
artifactory = Artifactory()
jenkins = Jenkins(artifactory, bitbucket)

## Setup Library
First, we create the repository for the library. In the real world, you'd e.g. click the "New Repository" menu entry on GitHub. We name the repo just `lib` to keep its name short.

In [ ]:
lib = bitbucket.create_repo('lib')

By default, a `master` branch will be created. To keep things simple, we just work with that branch

### Version introduction
Now it's time to introduce "versioning". CICD simulator helps tangling Conan versioning which is based on https://semver.org.

Each and every project (libraries, applications, ...) must have a version. Other projects may then "require" specific versions or version ranges of other projects.

E.g. An application may "require" a library names `lib` at version `1.2.3-pre.0`

### Setup LIBs version
So we shall spend our newly created library a version. The CICD simulator provides an according function to do so:

In [ ]:
lib_master = lib.checkout('master')
lib_master.set_version('1.0.0')

We then commit some file to our project and finally push the branch to the 'Git service'.

Pushing, automatically triggers a build at our buildmachine

In [ ]:
lib_master.commit_file('My cool file', 'The file content')
lib_master.push()

We now have the library project setup and already having created a first build artefact: `lib/1.0.0`. The version number `1.0.0` indicates a "stable release" as no pre-release tag is present.

That artefact is now stored in our (virtual) Artifactory, as can be checked by the following command

In [ ]:
artifactory.get_artifacts()

## Setup Application
Create Git repo for the application. We also set the version of this application.

In [ ]:
app = bitbucket.create_repo('app')
app_master = app.checkout('master')
app_master.set_version('0.1.0')

This application shall use the previously created library. Therefore, we need to setup 'requires', similar to what you'd do in a `conanfile.py`

In [ ]:
app_master.set_requires('lib/>1.0.0-0')

Finally, we push the Git master. This triggers a build at Jenkins. "Building" the application means that Jenkins executes a `conan install`. This, in turn, checks the availability of `lib` in the setup version, `>1.0.0-0` in this case.

The CICD simulator outputs according messages to indicate the actually found version of `lib` as well as the version of `application` that's finally published to Artifactory

In [ ]:
app_master.push()

As mentioned before, there should now be a `app` artifact. Let's check that:

In [ ]:
artifactory.get_artifacts()